In [0]:
!pip install --upgrade scikit-learn

In [0]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras as keras

In [0]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

# After executing the cell above, Drive
# files will be present in "/content/drive/My Drive".
!ls "/content/drive/My Drive/AML Task4/Task5"

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive
Feature_Extraction.m		     Task5.ipynb    train_eeg1.csv
Feature_Extraction_Test.m	     test_eeg1.csv  train_eeg2.csv
relEnergy_KurtosisFeatures.csv	     test_eeg2.csv  train_emg.csv
relEnergy_KurtosisFeatures_test.csv  test_emg.csv   train_labels.csv


In [0]:
from sklearn.preprocessing import robust_scale

def process_file(file_name):
  df=pd.read_csv(file_name, header=0)
  x=df.values[:,1:]
  x=robust_scale(x)
  x=np.expand_dims(x,axis=2)
  return x
#Input file names
x_train_file1 ="/content/drive/My Drive/AML Task4/Task5/train_eeg1.csv"
x_train_file2 ="/content/drive/My Drive/AML Task4/Task5/train_eeg2.csv"
x_train_file3 ="/content/drive/My Drive/AML Task4/Task5/train_emg.csv"
y_file        ="/content/drive/My Drive/AML Task4/Task5/train_labels.csv"
x_out_file1   ="/content/drive/My Drive/AML Task4/Task5/test_eeg1.csv"
x_out_file2   ="/content/drive/My Drive/AML Task4/Task5/test_eeg2.csv"
x_out_file3   ="/content/drive/My Drive/AML Task4/Task5/test_emg.csv"
#Import csv files as data frames
#Extract, from the data frames, the 2D arrays of values
x_train1 =process_file(x_train_file1)
x_train2 =process_file(x_train_file2)
x_train3 =process_file(x_train_file3)
x_out1   =process_file(x_out_file1)
x_out2   =process_file(x_out_file2)
x_out3   =process_file(x_out_file3)
df_y     =pd.read_csv(y_file, header=0)

# Extract, from the data frames, the 2D arrays of values
#np.expand_dims(x_train, axis=2)
#x_out = np.expand_dims(x_out, axis=2)
x_train_data = np.concatenate((x_train1, x_train2, x_train3),axis=2)
x_out_data = np.concatenate((x_out1, x_out2, x_out3),axis=2)
y_train_data = df_y.values[:, 1]

In [0]:
print('x train shape:',x_train_data.shape)
print('x out shape:',x_out_data.shape)
print('y train shape:',y_train_data.shape)

x train shape: (64800, 512, 3)
x out shape: (43200, 512, 3)
y train shape: (64800,)


# One hot encoding and normalization - OneHotEncoder doesn't work

In [0]:
#from sklearn.preprocessing import OneHotEncoder
#cat=[1, 2, 3]
#enc=OneHotEncoder(categories=cat);
#A=enc.fit_transform(y_train_data).toarray()


In [0]:
nb_classes = 3
train_labels = np.zeros((len(x_train_data), nb_classes)) #Creating of one-hot encodings representation
for i in range(len(y_train_data)):
    dummy = np.zeros((nb_classes))
    dummy[int(y_train_data[i])-1] = 1
    train_labels[i, :] = dummy



#x_train = np.expand_dims(x_train, axis=2)
#x_out = np.expand_dims(x_out, axis=2)
#Train test split
TEST_SIZE = 0.3
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x_train_data, train_labels,
                                                  test_size=TEST_SIZE, random_state=42)

x_train=x_train_data[:,:]
y_train=train_labels[:]
x_val=x_train_data[:21600,:]
y_val=train_labels[:21600]

# Neural Network

In [0]:
unique, counts = np.unique(y_train_data, return_counts=True)
print(unique)
print(counts)

[1 2 3]
[34114 27133  3553]


In [0]:
from tensorflow.keras.optimizers import SGD
from sklearn.metrics import balanced_accuracy_score as BMAC

class Metrics(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self._data = []

    def on_epoch_end(self, batch, logs={}):
        X_val, y_val = self.validation_data[0], self.validation_data[1]
        y_predict = np.asarray(model.predict(X_val))

        y_val = np.argmax(y_val, axis=1)
        y_predict = np.argmax(y_predict, axis=1)

        self._data.append({
            'val_bmac': BMAC(y_val, y_predict),
        })
        return

    def get_data(self):
        return self._data
      

In [0]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Conv2D, MaxPooling2D, Flatten, LSTM, Conv1D, GlobalAveragePooling1D, MaxPooling1D, GlobalMaxPooling1D, AveragePooling1D
from keras import regularizers
from keras.callbacks import ModelCheckpoint
import keras.backend as K
import os.path
import numpy as np
output_dir='/content/drive/My Drive/AML Task5/'

#class weights
unique, counts = np.unique(y_train_data, return_counts=True)
sum_counts=np.sum(counts)
class_weights = {0 : sum_counts/counts[0],
                1 : sum_counts/counts[1],
                2: sum_counts/counts[2]}

model = Sequential()
model.add(Conv1D(64, 10, activation='relu', input_shape=(512,3)))
model.add(MaxPooling1D(3))
model.add(Dropout(0.5))
model.add(Conv1D(128, 10, activation='relu'))
model.add(MaxPooling1D(3))
model.add(Dropout(0.5))
model.add(Conv1D(256, 5, activation='relu'))

model.add(GlobalAveragePooling1D())	
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(nb_classes, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])

#checkpointer = ModelCheckpoint(filepath=os.path.join(output_dir, 'Best_model.h5'),
#                               monitor='val_acc', verbose=1, save_best_only=True)
hist = model.fit(x_train, y_train, validation_data=(x_val, y_val), batch_size=128,
                 epochs=20, verbose=1, shuffle=True,class_weight = class_weights)


# Plot results

In [0]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

y_pred=model.predict(x_val)
y_pred=y_pred.argmax(1)+1
y_true=y_val.argmax(1)+1
conf_mat=confusion_matrix(y_true,y_pred)
print(conf_mat)

val_acc=[d['val_bmac'] for d in metrics.get_data()]

plt.plot(hist.history['categorical_accuracy'])
plt.plot(val_acc)
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()


Debuging

In [0]:
print(y_pred)
out=np.concatenate((indexes.reshape((-1,1)),predictions_class.reshape((-1,1))),axis=1)
print(out)
print(x_out.shape[0])
print(index2.shape)
print(predictions_class.shape)
unique, counts = np.unique(y_true, return_counts=True)
print(unique,counts)

# Calculate the transition and emission probabilities

In [0]:
P_transition=np.zeros((3,3))

for i in range(len(y_train_data)-1):
  k=y_train_data[i]-1
  l=y_train_data[i+1]-1
  P_transition[l,k]+=1 #Prob of transition from column to line
P_transition=np.divide(P_transition,P_transition.sum(axis=0,keepdims=True))
log_P_transition=np.log(P_transition)
print(P_transition)
  
from sklearn.metrics import confusion_matrix

y_pred=model.predict(x_val)
y_pred=y_pred.argmax(1)
y_true=y_val.argmax(1)
conf_matrix=confusion_matrix(y_true,y_pred)
P_emission=np.divide(conf_matrix,conf_matrix.sum(axis=1,keepdims=True))
log_P_emission=np.log(P_emission)
print(P_emission)# emmision from the line state  to the column state 

[[9.66757541e-01 3.25433973e-02 7.09259780e-02]
 [3.32131445e-02 9.58169019e-01 2.81452294e-04]
 [2.93143376e-05 9.28758339e-03 9.28792570e-01]]
[[0.93790036 0.05320285 0.0088968 ]
 [0.0209176  0.92869525 0.05038715]
 [0.00234329 0.02987698 0.96777973]]


In [0]:
def applyViterbi(x):
  y=model.predict(x)
  N=len(y)
  P=np.asarray([[0.33], [0.33], [0.33]])
  log_p=np.log(P)
  backpointers=-np.ones((N,nb_classes))
  obs=y.argmax(1)

  for i in range(N-1):
    o=obs[i+1]
    log_P_matrix=np.add(log_p,log_P_transition)
    log_p=log_P_matrix.max(1)+log_P_emission[:,o]
    backpointers[i+1,:]=log_P_matrix.argmax(1)
    #print(log_P_matrix.max(1))
    #print(log_p)
    #wait = input("PRESS ENTER TO CONTINUE.")  
  #backward pass
  y_viterbi=np.zeros((N,1))
  state=log_p.argmax()

  for i in range(N):
    #print(state)
    y_viterbi[N-i-1]=state+1
    state=int(backpointers[N-i-1,state])
  return y_viterbi

In [0]:
a=np.array([1, 2, 3])
b=np.array([[1, 2, 2],[3, 2, 2]])
np.add(a,b)

print(log_P_matrix)
print(log_P_emission[:,o])
print(backpointers)
wait = input("PRESS ENTER TO CONTINUE.")

In [0]:
log_p.shape
print(backpointers[:,:])
print(N)

# Other Aproach
Take $\alpha$ \* the output of the network +$ (1-\alpha) $\* transition probability


In [0]:
def applyFilter(x):
  y=model.predict(x)
  N=len(y)
  P0=np.zeros((1,3))
  y_combined=np.zeros(N)
  alpha=0.9
  for i in range(N):
    if i==0:
      p=P0
    else:
      p=P_transition[:,s].reshape((1,-1))
    P_combined=y[i,:]*alpha+(1-alpha)*p
    s=P_combined.argmax(1)
    #print(y[i,:])
    #print(P_combined)
    #print(p)
    #wait = input("PRESS ENTER TO CONTINUE.")
    y_combined[i]=s+1
  return y_combined
    
#backward pass

# Compare methods

In [0]:
y_pred=model.predict(x_val)
y_pred=y_pred.argmax(1)+1
y_true=y_val.argmax(1)+1

y_viterbi=applyViterbi(x_val)
y_combined=applyFilter(x_val)

conf1_matrix=confusion_matrix(y_true,y_pred)
conf2_matrix=confusion_matrix(y_true,y_viterbi)
conf3_matrix=confusion_matrix(y_true,y_combined)
unique, counts = np.unique(y_true, return_counts=True)
print('counts y_true\n',unique,counts)

unique, counts = np.unique(y_pred, return_counts=True)
print('counts y_pred\n',unique,counts)

unique, counts = np.unique(y_viterbi, return_counts=True)
print('counts y_viterbi\n',unique,counts)

unique, counts = np.unique(y_combined, return_counts=True)
print('counts y_combined\n',unique,counts)

print('Confusion matrix y_pred\n',conf1_matrix)
print('Confusion matrix y_viterbi\n',conf2_matrix)
print('Confusion matrix y_combined\n',conf3_matrix)

print('Score pred:',BMAC(y_true,y_pred))
print('Score viterbi:',BMAC(y_true,y_viterbi))
print('Score combined:',BMAC(y_true,y_combined))

counts y_true
 [1 2 3] [11240  8653  1707]
counts y_pred
 [1 2 3] [10727  8685  2188]
counts y_viterbi
 [1. 2. 3.] [10715  8979  1906]
counts y_combined
 [1. 2. 3.] [10793  8662  2145]
Confusion matrix y_pred
 [[10542   598   100]
 [  181  8036   436]
 [    4    51  1652]]
Confusion matrix y_viterbi
 [[10531   693    16]
 [  175  8231   247]
 [    9    55  1643]]
Confusion matrix y_combined
 [[10610   552    78]
 [  179  8060   414]
 [    4    50  1653]]
Score pred: 0.9447917788651702
Score viterbi: 0.9502199393279523
Score combined: 0.9479281954237392


# Make Predictions


In [0]:
def download_forecast(predictions,filename):
  from google.colab import files
  indexes=np.asarray(range(x_out_data.shape[0]))
  forecast_set=np.concatenate((indexes.reshape((-1,1)),predictions.reshape((-1,1))),axis=1)
  df=pd.DataFrame(forecast_set)
  path='/content/'+filename
  df.to_csv(path,sep=',',float_format='%s',index=False, header=['Id','y'])
  files.download(path)

In [0]:
#predict
predictions_proba = model.predict(x_out_data)
predictions_class=predictions_proba.argmax(1)+1
download_forecast(predictions_class,'forecast_simple.csv')

predictions_viterbi=applyViterbi(x_out_data)
download_forecast(predictions_viterbi,'forecast_viterbi.csv')